In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
mpl.rc('font', family='Malgun Gothic')
mpl.rc('axes', unicode_minus=False)
import folium
import json
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
from glob import glob
# 파일 갯수 확인
len(glob('./과제data/소상공인*.csv'))

17

In [68]:
all_shop = glob('./과제data/소상공인*.csv')

In [69]:
df = pd.read_csv(all_shop[1], sep='|')
df['상호명'] = df['상호명'].str.lower()

In [70]:
tmp[tmp['상호명'].str.find('롯데리아') != -1]

,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,...,건물관리번호,건물명,도로명주소,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도
2222,19956559,롯데리아평택안중점,평택안중점,Q,음식,Q07,패스트푸드,Q07A04,패스트푸드,I56199,...,4122025321102780015000005,NaN,경기도 평택시 안중읍 안중로 109-1,451884.0,17933.0,277,2,17,126.928617,36.988845
3016,20637991,롯데리아,당동지점,Q,음식,Q07,패스트푸드,Q07A04,패스트푸드,I56199,...,4141010100109210002002005,NaN,경기도 군포시 고산로 262,435010.0,15856.0,NaN,1,NaN,126.943597,37.346761
3148,20619175,롯데리아양주덕정점,양주덕정점,Q,음식,Q07,패스트푸드,Q07A04,패스트푸드,I56199,...,4163011000102080008012492,황금프라자,경기도 양주시 고암길 306-77,482060.0,11456.0,NaN,1,NaN,127.068638,37.836745
17277,20078050,롯데리아회룡역점,회룡역점,Q,음식,Q07,패스트푸드,Q07A04,패스트푸드,I56199,...,4115010200104250000015334,한승아파트,경기도 의정부시 평화로 374,480786.0,11704.0,112,2,116,127.048539,37.725282
18075,20394535,롯데리아,안양관양점,Q,음식,Q07,패스트푸드,Q07A04,패스트푸드,I56199,...,4117310200113840025003358,NaN,경기도 안양시 동안구 관악대로 291,431060.0,13940.0,NaN,1,NaN,126.955321,37.403154
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
587508,16595031,롯데리아,미사역점,Q,음식,Q07,패스트푸드,Q07A04,패스트푸드,I56199,...,4145010900110790002000001,성산타워,경기도 하남시 미사강변중앙로 218,465150.0,12913.0,NaN,NaN,NaN,127.189648,37.566392
592981,16952724,롯데리아,야당역점,Q,음식,Q07,패스트푸드,Q07A04,패스트푸드,I56199,...,4148010800110720000000001,NaN,경기도 파주시 경의로 1092,413200.0,10908.0,NaN,NaN,114,126.760073,37.714906
593058,16958185,롯데리아,김포풍무점,Q,음식,Q07,패스트푸드,Q07A04,패스트푸드,I56199,...,4157010700101150008022545,NaN,경기도 김포시 풍무로 122,415808.0,10114.0,NaN,NaN,NaN,126.722794,37.606279
594198,16953986,롯데리아,화도DT점,Q,음식,Q07,패스트푸드,Q07A04,패스트푸드,I56199,...,4136025622103580006002730,NaN,경기도 남양주시 화도읍 먹갓로28번길 77-4,472846.0,12171.0,2,2,1,127.284175,37.657901


In [71]:
tmp[(tmp['도로명주소'].str.contains('경기도 광명시')) & (tmp['상호명'].str.contains('롯데리아')) ]

,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,...,건물관리번호,건물명,도로명주소,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도
333094,19961669,롯데리아,광명하안점,Q,음식,Q07,패스트푸드,Q07A04,패스트푸드,I56199,...,4121010300100340008012304,NaN,경기도 광명시 범안로 1045,423060.0,14248.0,NaN,1,NaN,126.878829,37.462419
408624,12070752,롯데리아,광명역사점,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,I56111,...,4121010600102760008014873,고속철도광명역,경기도 광명시 광명역로 21,423090.0,14347.0,NaN,1,NaN,126.884837,37.416369


In [72]:
len(tmp[(tmp['상호명'].str.contains('케이에프시')) | (tmp['상호명'].str.contains('kfc')) | (tmp['상호명'].str.contains('케이에프씨'))])

124

In [73]:
import cartogram.fillter as ft


In [74]:
c_d = ft.fillter(all_shop)

In [75]:
c_p = pd.DataFrame(c_d)

In [76]:
c_p.rename({0:'ID', 1:'버거킹', 2: '맥도날드', 3:'KFC', 4:'롯데리아', 5:'맘스터치'},axis=1 ,inplace=True)

In [77]:
c_p['버거지수'] = (c_p['버거킹'] + c_p['맥도날드'] + c_p['KFC']) / (c_p['롯데리아'] + c_p['맘스터치'])

In [78]:
c_p

,ID,버거킹,맥도날드,KFC,롯데리아,맘스터치,버거지수
0,홍천,4,3,0,3,1,1.750000
1,강릉,1,5,0,5,4,0.666667
2,영월,0,0,0,0,0,NaN
3,원주,1,13,0,13,4,0.823529
4,삼척,0,0,0,0,1,0.000000
...,...,...,...,...,...,...,...
245,괴산,0,1,0,1,0,1.000000
246,증평,0,1,0,1,1,0.500000
247,영동,0,0,0,0,0,NaN
248,단양,0,0,0,0,0,NaN


In [79]:
c_p[c_p['버거지수'] == np.inf]

,ID,버거킹,맥도날드,KFC,롯데리아,맘스터치,버거지수
9,동해,0,0,1,0,0,inf


In [80]:
c_p.replace([np.inf], 0,inplace=True)

In [81]:
c_p = c_p.fillna(0)

In [82]:
draw_korea_raw = pd.read_excel('./과제data/draw_korea_raw(수정).xlsx')
draw_korea_raw_stacked = pd.DataFrame(draw_korea_raw.stack())
draw_korea_raw_stacked.reset_index(inplace=True)
draw_korea_raw_stacked.rename(columns={'level_0' : 'y', 'level_1' : 'x', 0: 'ID'}, inplace=True)
draw_korea = draw_korea_raw_stacked
draw_korea.loc[draw_korea['ID'] == '인천 남구', 'ID'] = '인천 미추홀'

In [83]:
c_p = pd.merge(c_p, draw_korea, how='left', on=['ID'])
c_p.head()

,ID,버거킹,맥도날드,KFC,롯데리아,맘스터치,버거지수,y,x
0,홍천,4,3,0,3,1,1.750000,2,8
1,강릉,1,5,0,5,4,0.666667,3,11
2,영월,0,0,0,0,0,0.000000,7,10
3,원주,1,13,0,13,4,0.823529,5,10
4,삼척,0,0,0,0,1,0.000000,5,11


In [84]:
c_p.to_csv('./static/data/b_p.csv')

In [85]:
df = pd.read_csv('./static/data/b_p.csv')